In [1]:
# 📌 Live SIP Recommendation Engine (using mfapi.in and Gemini API)

import requests
import pandas as pd
from datetime import datetime
import google.generativeai as genai
import random
import os

# Import userdata for Google Colab secrets
try:
    from google.colab import userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

# -------------------------------
# 1. API Configuration
# -------------------------------

# --- IMPORTANT ---
# This script is now configured to use Google Colab's secrets manager.
# Make sure you have a secret named 'GEMINI_API_KEY' with your API key.
try:
    if IS_COLAB:
        print("Running in Google Colab. Attempting to fetch API key from secrets...")
        # Fetch the key from Colab secrets
        api_key = userdata.get('GEMINI_API_KEY')
        if not api_key:
            raise ValueError("Secret 'GEMINI_API_KEY' not found or is empty in Colab secrets.")
    else:
        print("Not in Colab. Attempting to fetch API key from environment variables...")
        # Fallback to environment variables if not in Colab
        api_key = os.getenv("GEMINI_API_KEY")
        if not api_key:
            raise ValueError("GEMINI_API_KEY environment variable not found.")

    genai.configure(api_key=api_key)
    print("✅ Gemini API configured successfully.")

except Exception as e:
    print(f"❌ Error configuring Gemini API: {e}")
    if IS_COLAB:
        print("👉 Please go to the '🔑' (Secrets) tab in the left sidebar of Colab and add a new secret with the name 'GEMINI_API_KEY' and your API key as the value.")
    else:
        print("👉 Please make sure you have the 'google-generativeai' library installed (`pip install google-generativeai`) and your API key is set as an environment variable.")
    # Exit or handle the error gracefully if the API key is not configured
    api_key = None # Ensure api_key is None if configuration fails

# Initialize the Gemini Model if the API key was successfully configured
if api_key:
    model = genai.GenerativeModel("gemini-1.5-flash")
else:
    model = None


# -------------------------------
# 2. Helper Functions
# -------------------------------

def get_all_funds():
    """
    Fetch a list of all available mutual fund schemes from mfapi.in.
    """
    try:
        url = "https://api.mfapi.in/mf"
        res = requests.get(url, timeout=10)
        res.raise_for_status()  # Will raise an HTTPError for bad responses (4xx or 5xx)
        return res.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching fund list from mfapi.in: {e}")
        return None

def suggest_sip_amount(annual_income, risk_level):
    """
    Suggest a monthly SIP amount based on income and risk appetite.
    """
    risk_map = {"low": 0.10, "moderate": 0.15, "aggressive": 0.20}
    # Default to 'moderate' if risk_level is unrecognized
    investment_percentage = risk_map.get(risk_level.lower(), 0.15)
    # The suggested amount is at least ₹500
    return max(500, int((annual_income * investment_percentage) / 12))

def get_gemini_recommendation(annual_income, horizon_years, risk_level):
    """
    Fetches a sample of funds and uses the Gemini API to generate a recommendation.
    """
    # Check if the model was initialized
    if not model:
        return "Cannot generate recommendation because the Gemini API is not configured."

    print("\nFetching live fund data...")
    all_funds = get_all_funds()
    if not all_funds:
        return "Could not generate recommendations because the fund list could not be fetched."

    print(f"Found {len(all_funds)} funds. Filtering based on risk level: '{risk_level}'...")

    # --- Intelligent Filtering & Sampling ---
    if risk_level == "aggressive":
        keywords = ['Small Cap', 'Midcap', 'Sectoral', 'Thematic', 'Technology', 'Infra']
    elif risk_level == 'moderate':
        keywords = ['Flexi Cap', 'Large & Mid Cap', 'Multi Cap', 'Hybrid', 'Balanced Advantage', 'Value']
    else:  # 'low' risk
        keywords = ['Debt', 'Liquid', 'Gilt', 'Conservative', 'Corporate Bond', 'Short Duration']

    filtered_funds = [f for f in all_funds if any(k.lower() in f['schemeName'].lower() for k in keywords)]

    if len(filtered_funds) < 15:
        sample_source = all_funds
    else:
        sample_source = filtered_funds

    sample_size = min(50, len(sample_source))
    fund_sample = random.sample(sample_source, sample_size)
    print(f"Providing a sample of {len(fund_sample)} funds to the Gemini API for analysis...")

    # --- Build Prompt for Gemini ---
    prompt = f"""
    You are an expert AI financial advisor. Your task is to recommend suitable mutual funds for a user based on their profile and a provided list of live funds.

    **User Profile:**
    - Annual Income: ₹{annual_income:,.0f}
    - Investment Horizon: {horizon_years} years
    - Risk Level: {risk_level.title()}

    **Instructions:**
    1.  Analyze the user's profile carefully. A {horizon_years}-year horizon is long-term. A '{risk_level}' risk tolerance means they are willing to accept a certain level of market fluctuation for potential returns.
    2.  From the **List of Available Mutual Funds** below, recommend exactly 3 suitable funds for a Systematic Investment Plan (SIP).
    3.  For each recommended fund, provide:
        - The full Scheme Name and its Scheme Code.
        - A clear, concise explanation (2-3 sentences) of WHY this specific fund is a good fit for the user's profile (risk, horizon, and potential goals).
        - The fund's primary category (e.g., Small Cap, Flexi Cap, Balanced Advantage).
    4.  Structure your response in a clean, easy-to-read markdown format.
    5.  **Crucially, do not recommend any funds that are not in the list provided below.**

    **List of Available Mutual Funds (JSON format):**
    {fund_sample}

    Please provide your detailed recommendation now.
    """
    print("Generating recommendation with Gemini AI...")
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"An error occurred while communicating with the Gemini API: {e}"

# -------------------------------
# 3. User Inputs
# -------------------------------
annual_income = 900000   # e.g. ₹9,00,000
horizon_years = 10       # e.g. 10 years
risk_level = "moderate"  # Options: low / moderate / aggressive

# -------------------------------
# 4. SIP Calculation & Recommendation
# -------------------------------
monthly_sip = suggest_sip_amount(annual_income, risk_level)

print("-" * 50)
print(f"📊 Live SIP Recommendation ({datetime.now().strftime('%Y-%m-%d %H:%M:%S')})")
print("-" * 50)
print(f"👤 User Profile:")
print(f"   - Annual Income: ₹{annual_income:,.0f}")
print(f"   - Horizon: {horizon_years} years")
print(f"   - Risk Level: {risk_level.title()}")
print(f"\n💡 Suggested Monthly SIP: ₹{monthly_sip:,.0f}")
print("-" * 50)

# Generate and print the AI-powered recommendation
recommendation = get_gemini_recommendation(annual_income, horizon_years, risk_level)
print("\n🤖 Gemini AI Recommendation:\n")
print(recommendation)
print("-" * 50)

Running in Google Colab. Attempting to fetch API key from secrets...
✅ Gemini API configured successfully.
--------------------------------------------------
📊 Live SIP Recommendation (2025-09-11 07:09:23)
--------------------------------------------------
👤 User Profile:
   - Annual Income: ₹900,000
   - Horizon: 10 years
   - Risk Level: Moderate

💡 Suggested Monthly SIP: ₹11,250
--------------------------------------------------

Fetching live fund data...
Found 37122 funds. Filtering based on risk level: 'moderate'...
Providing a sample of 50 funds to the Gemini API for analysis...
Generating recommendation with Gemini AI...

🤖 Gemini AI Recommendation:

Based on your annual income of ₹900,000, a 10-year investment horizon, and moderate risk tolerance, I recommend the following three mutual funds for a Systematic Investment Plan (SIP):

**1. HSBC Balanced Advantage Fund - Direct Growth (Scheme Code: 151129)**

* **Primary Category:** Balanced Advantage
* **Why it's suitable:** This

In [2]:
# 📌 Live SIP Recommendation Engine (using mfapi.in and Gemini API)

import requests
import pandas as pd
from datetime import datetime
import google.generativeai as genai
import numpy as np
import os

# Import userdata for Google Colab secrets
try:
    from google.colab import userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

# -------------------------------
# 1. API Configuration
# -------------------------------
try:
    if IS_COLAB:
        print("Running in Google Colab. Attempting to fetch API key from secrets...")
        api_key = userdata.get('GEMINI_API_KEY')
        if not api_key:
            raise ValueError("Secret 'GEMINI_API_KEY' not found or is empty in Colab secrets.")
    else:
        print("Not in Colab. Attempting to fetch API key from environment variables...")
        api_key = os.getenv("GEMINI_API_KEY")
        if not api_key:
            raise ValueError("GEMINI_API_KEY environment variable not found.")

    genai.configure(api_key=api_key)
    print("✅ Gemini API configured successfully.")

except Exception as e:
    print(f"❌ Error configuring Gemini API: {e}")
    if IS_COLAB:
        print("👉 Please add your GEMINI_API_KEY in Colab secrets.")
    else:
        print("👉 Please set GEMINI_API_KEY as an environment variable.")
    api_key = None

if api_key:
    model = genai.GenerativeModel("gemini-1.5-flash")
else:
    model = None


# -------------------------------
# 2. Helper Functions
# -------------------------------

def get_all_funds():
    try:
        url = "https://api.mfapi.in/mf"
        res = requests.get(url, timeout=10)
        res.raise_for_status()
        return res.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching fund list: {e}")
        return None

def suggest_sip_amount(annual_income, risk_level):
    risk_map = {"low": 0.10, "moderate": 0.15, "aggressive": 0.20}
    investment_percentage = risk_map.get(risk_level.lower(), 0.15)
    return max(500, int((annual_income * investment_percentage) / 12))

# ---- Fund Analysis Functions ----
def fetch_nav_history(scheme_code):
    try:
        url = f"https://api.mfapi.in/mf/{scheme_code}"
        res = requests.get(url, timeout=10).json()
        return res.get("data", [])
    except:
        return []

def calculate_cagr(nav_data, years=3):
    try:
        if len(nav_data) < years * 250:
            return None
        end_nav = float(nav_data[0]['nav'])
        start_nav = float(nav_data[-(years*250)]['nav'])
        return (end_nav/start_nav) ** (1/years) - 1
    except:
        return None

def calculate_volatility(nav_data, days=365):
    try:
        nav_values = [float(d['nav']) for d in nav_data[:days]]
        daily_returns = np.diff(nav_values) / nav_values[:-1]
        return np.std(daily_returns)
    except:
        return None

def pick_top_funds(funds, top_n=50):
    scored_funds = []
    for f in funds[:200]:  # Limit to first 200 funds for performance
        scheme_code = f['schemeCode']
        nav_data = fetch_nav_history(scheme_code)
        if not nav_data:
            continue
        cagr = calculate_cagr(nav_data, years=3)
        risk = calculate_volatility(nav_data, days=365)
        if cagr and risk and risk > 0:
            score = cagr / risk
            scored_funds.append({"fund": f, "cagr": cagr, "risk": risk, "score": score})

    scored_funds.sort(key=lambda x: x['score'], reverse=True)
    top_funds = [s["fund"] for s in scored_funds[:top_n]]
    return top_funds


# -------------------------------
# Gemini Recommendation
# -------------------------------

def get_gemini_recommendation(annual_income, horizon_years, risk_level):
    if not model:
        return "Gemini API is not configured."

    print("\nFetching live fund data...")
    all_funds = get_all_funds()
    if not all_funds:
        return "Could not fetch fund list."

    print(f"Found {len(all_funds)} funds. Filtering based on risk level: '{risk_level}'...")

    if risk_level == "aggressive":
        keywords = ['Small Cap', 'Midcap', 'Sectoral', 'Thematic', 'Technology', 'Infra']
    elif risk_level == 'moderate':
        keywords = ['Flexi Cap', 'Large & Mid Cap', 'Multi Cap', 'Hybrid', 'Balanced Advantage', 'Value']
    else:
        keywords = ['Debt', 'Liquid', 'Gilt', 'Conservative', 'Corporate Bond', 'Short Duration']

    filtered_funds = [f for f in all_funds if any(k.lower() in f['schemeName'].lower() for k in keywords)]
    if len(filtered_funds) < 15:
        sample_source = all_funds
    else:
        sample_source = filtered_funds

    print("Ranking funds by return/risk ratio...")
    fund_sample = pick_top_funds(sample_source, 50)
    print(f"Providing top {len(fund_sample)} funds to Gemini API for analysis...")

    prompt = f"""
    You are an expert AI financial advisor. Recommend 3 mutual funds from the provided list.

    **User Profile:**
    - Annual Income: ₹{annual_income:,.0f}
    - Investment Horizon: {horizon_years} years
    - Risk Level: {risk_level.title()}

    **Instructions:**
    1. Recommend exactly 3 funds.
    2. For each: give Scheme Name, Scheme Code, Category, and 2-3 line explanation.
    3. Use only funds from this list:
    {fund_sample}
    """

    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Error with Gemini API: {e}"


# -------------------------------
# 3. User Inputs (Dynamic)
# -------------------------------
try:
    annual_income = int(input("Enter your Annual Income (₹): "))
except:
    annual_income = 900000  # default
    print("⚠️ Invalid input, using default ₹9,00,000")

try:
    horizon_years = int(input("Enter your Investment Horizon (in years): "))
except:
    horizon_years = 10  # default
    print("⚠️ Invalid input, using default 10 years")

risk_level = input("Enter your Risk Level (low / moderate / aggressive): ").strip().lower()
if risk_level not in ["low", "moderate", "aggressive"]:
    risk_level = "moderate"
    print("⚠️ Invalid input, defaulting to 'moderate'")


# -------------------------------
# 4. SIP Calculation & Recommendation
# -------------------------------
monthly_sip = suggest_sip_amount(annual_income, risk_level)

print("-" * 50)
print(f"📊 Live SIP Recommendation ({datetime.now().strftime('%Y-%m-%d %H:%M:%S')})")
print("-" * 50)
print(f"👤 User Profile:")
print(f"   - Annual Income: ₹{annual_income:,.0f}")
print(f"   - Horizon: {horizon_years} years")
print(f"   - Risk Level: {risk_level.title()}")
print(f"\n💡 Suggested Monthly SIP: ₹{monthly_sip:,.0f}")
print("-" * 50)

recommendation = get_gemini_recommendation(annual_income, horizon_years, risk_level)
print("\n🤖 Gemini AI Recommendation:\n")
print(recommendation)
print("-" * 50)


Running in Google Colab. Attempting to fetch API key from secrets...
✅ Gemini API configured successfully.
Enter your Annual Income (₹): 2000000
Enter your Investment Horizon (in years): 20
Enter your Risk Level (low / moderate / aggressive): low
--------------------------------------------------
📊 Live SIP Recommendation (2025-09-11 09:30:04)
--------------------------------------------------
👤 User Profile:
   - Annual Income: ₹2,000,000
   - Horizon: 20 years
   - Risk Level: Low

💡 Suggested Monthly SIP: ₹16,666
--------------------------------------------------

Fetching live fund data...
Found 37122 funds. Filtering based on risk level: 'low'...
Ranking funds by return/risk ratio...


/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:218: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:175: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:210: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/tmp/ipython-input-272365764.py:90: RuntimeWarning: invalid value encountered in divide
  daily_returns = np.diff(nav_values) / nav_values[:-1]
/tmp/ipython-input-272365764.py:90: RuntimeWarning: divide by zero encountered in divide
  daily_returns = np.diff(nav_values) / nav_values[:-1]


Providing top 50 funds to Gemini API for analysis...

🤖 Gemini AI Recommendation:

Given your annual income of ₹2,000,000, a 20-year investment horizon, and low-risk preference, I recommend the following three liquid funds for diversification within this low-risk category.  Liquid funds are suitable for your low-risk profile and short-term needs, providing a safe haven for your capital.  However, note that returns are generally low compared to other asset classes. Consider incorporating other asset classes for long-term growth in a properly diversified portfolio.

**Recommendation:**


1. **Scheme Name:** Aditya Birla Sun Life Liquid Fund - Growth
   **Scheme Code:** 100047
   **Category:** Liquid Fund
   **Explanation:**  A well-established fund with a proven track record of stability and consistent returns.  It's a good core holding for your low-risk portfolio.  Its large size indicates robust management and liquidity.

2. **Scheme Name:** HDFC Liquid Fund - Growth Plan
   **Scheme C